In [14]:
import wikipedia
from langchain.agents import tool

wikipedia.set_lang("pt")

In [15]:
@tool
def busca_wikipedia(query: str):
    """Busca dados no wikipedia e retorna resumos de páginas para a query"""
    titulo_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulo_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f"Título: {titulo}\nResumo: {wiki_page.summary}")
        except:
            pass
    if not resumos:
        return "Busca não teve retorno"
    else:
        return "\n\n".join(resumos)

In [16]:
busca_wikipedia.invoke({"query": "Teste de software"})

/Users/jonatasmartins/.pyenv/versions/3.11.5/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/jonatasmartins/.pyenv/versions/3.11.5/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


"Título: Teste de software\nResumo: O teste do software  é a investigação do software a fim de fornecer informações sobre sua qualidade em relação ao contexto em que ele deve operar, se relaciona com o conceito de verificação e validação. Isso inclui o processo de utilizar o produto para encontrar seus defeitos.\nO teste é um processo realizado pelo testador de software, que permeia outros processos da engenharia de software, e que envolve ações que vão do levantamento de requisitos até a execução do teste propriamente dito.\n\nTítulo: Software\nResumo: Software (pronúncia em inglês: ['sɔftwɛəɹ]) é uma coleção de programas que dizem a um computador como executar tarefas específicas. Isso contrasta com o hardware, a partir do qual o sistema é construído e que realmente executa o trabalho.\nSoftware é o termo usado para se referir a uma sequência de instruções executadas em um dispositivo informático, como um computador ou máquina semelhante. Ele é um produto da engenharia de software e 

In [17]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente que sabe sobre teste de software"),
    ("user", "{input}")
])

chat = ChatOpenAI()

tools = [convert_to_openai_function(busca_wikipedia)]

chain = prompt | chat.bind(functions=tools)

In [18]:
chain.invoke({"input": "olá"})

AIMessage(content='Olá! Como posso te ajudar hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 70, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c133f2cd-2ef1-4dc8-8620-3f9f04d51c15-0', usage_metadata={'input_tokens': 70, 'output_tokens': 13, 'total_tokens': 83, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [20]:
resultado = chain.invoke({"input": "O que é teste de software?"})

print(resultado)

content='' additional_kwargs={'function_call': {'arguments': '{"query":"teste de software"}', 'name': 'busca_wikipedia'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 75, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None} id='run-30ac04d2-aa68-4fb2-80a0-652934baf693-0' usage_metadata={'input_tokens': 75, 'output_tokens': 19, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
